Requirement: Start looking at the compensation data set and start to figure out how to put the two data sources together.  
Source: Compensation data, WRDS data.
And I chose to use jupyter notebook.

## 1. Preparation
Preparation work includes installing modules and importing libraries (i.e., numpy, pandas, datetime, wrds) that I tended to use during coding.

### 1.1 Install 'wrds' module

In [1]:
pip install wrds

Note: you may need to restart the kernel to use updated packages.


### 1.2 Import libraries

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import wrds

### 1.3 Look at basic information about 'wrds'

In [3]:
help(wrds)

Help on package wrds:

NAME
    wrds

DESCRIPTION
    WRDS Python Data Access Library
    
    WRDS-Py is a library for extracting data from WRDS data sources and getting it into Pandas.
    
        >>> import wrds
        >>> db = wrds.Connection()
        >>> db.list_libraries()
        ['aha', 'aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', ...]
        >>> db.list_tables(library='crsp')
        ['acti', 'asia', 'asib', 'asic', 'asio', 'asix', 'bmdebt', 'bmheader', ...]
        >>> data = db.raw_sql('SELECT * FROM crsp.stocknames', index_col='permno')
        >>> data.head()
                 permco      namedt   nameenddt     cusip    ncusip ticker      permno
        10000.0  7952.0  1986-01-07  1987-06-11  68391610  68391610  OMFGA
        10001.0  7953.0  1986-01-09  1993-11-21  36720410  39040610   GFGC
        10001.0  7953.0  1993-11-22  2008-02-04  36720410  29274A10   EWST
        10001.0  7953.0  2008-02-05  2009-08-03  36720410  29274A20   EWST
        10001.0  7953

### 1.4 Get connected with wrds using my wrds account

In [4]:
conn=wrds.Connection()

Enter your WRDS username [minqi]:liuminqi
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [5]:
conn.list_libraries()

['aha',
 'aha_sample',
 'ahasamp',
 'audit',
 'auditsmp',
 'bank',
 'blab',
 'block',
 'block_all_new',
 'boardex',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'centris',
 'ciq',
 'ciqsamp',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_execucomp_new',
 'comp_global',
 'comp_global_daily',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_snapshot',
 'compseg',
 'compsegd',
 'compsnap',
 'comscore',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_general_new',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'crsp',
 'crsp_a_stock',
 'crsp_a_stock_new',
 'crspa',
 'crspm',
 'crspq',
 'crspsamp',
 'csmar',
 'dealscan',
 'djones',
 'dmef',
 'doe',
 'emdb',
 'etfg',
 'etfg_samp',

## 2. WRDS data

I thought it might be interesting to see association between financial data (e.g., net income, sales, etc.) and characteristics of CD&A section. Hence, I downloaded relevant data from 'Compustat Daily Updates - Fundamentals Annual'. 

### 2.1 Obtain wrds data using sql query
Common criteria for Compustat Fundamentals Annual (`funda`):
 - consol = "C" (fully consolidated)
 - indfmt = "INDL" (industrial format)
 - datafmt = "STD" (standard format)
 - posrc = "D" (domestic; USA and Canada)
 
 Additionally, there is another way to obtain wrds data:  
 
 comp = conn.get_table(library = 'comp', 
                   table ='funda', 
                       columns=['cik', 'conm','fyear', 'ni','sale', 'at','lt'])
But I felt that using sql was much more convenient.

In [6]:
comp=conn.raw_sql("""
                  SELECT cik,conm,fyear,ni,sale,at,lt
                  FROM comp.funda
                  WHERE (fyear between '2006' and '2016')
                  AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
                  """)
comp.head()               

,cik,conm,fyear,ni,sale,at,lt
0,0000002230,ADAMS DIVERSIFIED EQUITY FD,2006.0,NaN,NaN,NaN,NaN
1,0000001750,AAR CORP,2006.0,58.660,1061.169,1067.633,573.390
2,0000061478,ADC TELECOMMUNICATIONS INC,2006.0,65.700,1281.900,1611.400,737.900
3,0000319126,AFP IMAGING CORP,2006.0,1.005,24.998,14.341,3.672
4,0000730469,ALPHARMA INC -CL A,2006.0,82.544,653.828,927.239,203.240


One problem of this step is that it always took too long to get the dataframe. I have tried to search for more efficient methods but failed.

### 2.2 Check basic infomation about the comp dataframe

In [7]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123581 entries, 0 to 123580
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cik     95818 non-null   object 
 1   conm    123581 non-null  object 
 2   fyear   123581 non-null  float64
 3   ni      97603 non-null   float64
 4   sale    97599 non-null   float64
 5   at      98138 non-null   float64
 6   lt      97975 non-null   float64
dtypes: float64(5), object(2)
memory usage: 6.6+ MB


### 2.3 Transform type of 'fyear' 

In [8]:
comp['fyear']=comp['fyear'].astype(int)
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123581 entries, 0 to 123580
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cik     95818 non-null   object 
 1   conm    123581 non-null  object 
 2   fyear   123581 non-null  int64  
 3   ni      97603 non-null   float64
 4   sale    97599 non-null   float64
 5   at      98138 non-null   float64
 6   lt      97975 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 6.6+ MB


## 3. Import compensatin excel
The compensation excel was provided by Professor Greg and was located in the predefined path and renamed as SEC.xlsx.

### 3.1 Read excel, transform digit of cik.

In [9]:
sec=pd.read_excel('SEC.xlsx', index_col=False,converters={'CIK':'{:0>10}'.format})
sec.head()

/Users/minqi/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,CIK,CIK_10,Company Name,Form Type,Date Filed,Year,Duplicate CIK-date?,Duplicate CIK-year,Direct Link to Filing on SEC website,Downloaded Filename,...,Processed CD&A .txt File,Hand checked?,# of Images in CD&A Section,# of Tables in CD&A Section,# of Words in CD&A Section (including tabular content--old version),# of Sentences in CD&A (including tabular content--old version),# of Sentences with FLS (including tabular content--old version),# of Words in CD&A Section,# of Sentences in CD&A,# of Sentences with FLS
0,0001292653,1193125,Neutral Tandem Inc,DEF 14A,2010-04-22,2010,NaN,NaN,http://www.sec.gov/Archives/edgar/data/1292653...,1292653_2010-04-22.htm,...,1292653_2010-04-22.txt,Yes,0.0,3.0,9625.0,444.0,162.0,9035.0,402.0,149.0
1,0001013934,950123,STRAYER EDUCATION INC,DEF 14A,2011-03-29,2011,NaN,NaN,http://www.sec.gov/Archives/edgar/data/1013934...,1013934_2011-03-29.htm,...,1013934_2011-03-29.txt,Yes,0.0,0.0,3153.0,118.0,63.0,562.0,15.0,7.0
2,0000854775,1193125,DIGI INTERNATIONAL INC,DEF 14A,2012-12-05,2012,NaN,NaN,http://www.sec.gov/Archives/edgar/data/854775/...,854775_2012-12-05.htm,...,854775_2012-12-05.txt,Yes,1.0,5.0,5363.0,240.0,69.0,3819.0,162.0,56.0
3,0001158463,950123,JETBLUE AIRWAYS CORP,DEF 14A,2011-04-14,2011,NaN,NaN,http://www.sec.gov/Archives/edgar/data/1158463...,1158463_2011-04-14.htm,...,1158463_2011-04-14.txt,Yes,1.0,4.0,5133.0,220.0,81.0,4192.0,174.0,67.0
4,0000059558,950159,LINCOLN NATIONAL CORP,DEF 14A,2013-04-12,2013,NaN,NaN,http://www.sec.gov/Archives/edgar/data/59558/0...,59558_2013-04-12.htm,...,59558_2013-04-12.txt,Yes,11.0,2.0,8822.0,369.0,113.0,6398.0,253.0,93.0


### 3.2 Choose useful columns

In [10]:
sec1=sec.iloc[:,[0,2,5,-8,-7,-6,-5,-4,-3,-2,-1]]
sec1.head()

,CIK,Company Name,Year,# of Images in CD&A Section,# of Tables in CD&A Section,# of Words in CD&A Section (including tabular content--old version),# of Sentences in CD&A (including tabular content--old version),# of Sentences with FLS (including tabular content--old version),# of Words in CD&A Section,# of Sentences in CD&A,# of Sentences with FLS
0,0001292653,Neutral Tandem Inc,2010,0.0,3.0,9625.0,444.0,162.0,9035.0,402.0,149.0
1,0001013934,STRAYER EDUCATION INC,2011,0.0,0.0,3153.0,118.0,63.0,562.0,15.0,7.0
2,0000854775,DIGI INTERNATIONAL INC,2012,1.0,5.0,5363.0,240.0,69.0,3819.0,162.0,56.0
3,0001158463,JETBLUE AIRWAYS CORP,2011,1.0,4.0,5133.0,220.0,81.0,4192.0,174.0,67.0
4,0000059558,LINCOLN NATIONAL CORP,2013,11.0,2.0,8822.0,369.0,113.0,6398.0,253.0,93.0


### 3.3 Transform column names into lower case for the convenience when coding

In [11]:
sec1.columns=map(str.lower,sec1.columns)
sec1.head()

,cik,company name,year,# of images in cd&a section,# of tables in cd&a section,# of words in cd&a section (including tabular content--old version),# of sentences in cd&a (including tabular content--old version),# of sentences with fls (including tabular content--old version),# of words in cd&a section,# of sentences in cd&a,# of sentences with fls
0,0001292653,Neutral Tandem Inc,2010,0.0,3.0,9625.0,444.0,162.0,9035.0,402.0,149.0
1,0001013934,STRAYER EDUCATION INC,2011,0.0,0.0,3153.0,118.0,63.0,562.0,15.0,7.0
2,0000854775,DIGI INTERNATIONAL INC,2012,1.0,5.0,5363.0,240.0,69.0,3819.0,162.0,56.0
3,0001158463,JETBLUE AIRWAYS CORP,2011,1.0,4.0,5133.0,220.0,81.0,4192.0,174.0,67.0
4,0000059558,LINCOLN NATIONAL CORP,2013,11.0,2.0,8822.0,369.0,113.0,6398.0,253.0,93.0


### 3.4 Transform all the company names into upper case to make company names consistent with those in wrds data.

In [12]:
sec1['company name']=sec1['company name'].str.upper()
sec1.head()

<ipython-input-12-a69ed2318fbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sec1['company name']=sec1['company name'].str.upper()


,cik,company name,year,# of images in cd&a section,# of tables in cd&a section,# of words in cd&a section (including tabular content--old version),# of sentences in cd&a (including tabular content--old version),# of sentences with fls (including tabular content--old version),# of words in cd&a section,# of sentences in cd&a,# of sentences with fls
0,0001292653,NEUTRAL TANDEM INC,2010,0.0,3.0,9625.0,444.0,162.0,9035.0,402.0,149.0
1,0001013934,STRAYER EDUCATION INC,2011,0.0,0.0,3153.0,118.0,63.0,562.0,15.0,7.0
2,0000854775,DIGI INTERNATIONAL INC,2012,1.0,5.0,5363.0,240.0,69.0,3819.0,162.0,56.0
3,0001158463,JETBLUE AIRWAYS CORP,2011,1.0,4.0,5133.0,220.0,81.0,4192.0,174.0,67.0
4,0000059558,LINCOLN NATIONAL CORP,2013,11.0,2.0,8822.0,369.0,113.0,6398.0,253.0,93.0


### 3.5 Look at information about compensation data

In [13]:
sec1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 11 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   cik                                                                  14994 non-null  object 
 1   company name                                                         14994 non-null  object 
 2   year                                                                 14994 non-null  int64  
 3   # of images in cd&a section                                          13434 non-null  float64
 4   # of tables in cd&a section                                          13434 non-null  float64
 5   # of words in cd&a section (including tabular content--old version)  13434 non-null  float64
 6   # of sentences in cd&a (including tabular content--old version)      100 non-null    float64
 7   # of

## 4. Merge WRDS data with Executive Compensation Data

### 4.1 I used left join to merge wrds data with compensation data on cik and year.


In [14]:
merge=pd.merge(sec1,comp,how='left',left_on=['cik','year'],right_on=['cik','fyear'])
merge.head()

,cik,company name,year,# of images in cd&a section,# of tables in cd&a section,# of words in cd&a section (including tabular content--old version),# of sentences in cd&a (including tabular content--old version),# of sentences with fls (including tabular content--old version),# of words in cd&a section,# of sentences in cd&a,# of sentences with fls,conm,fyear,ni,sale,at,lt
0,0001292653,NEUTRAL TANDEM INC,2010,0.0,3.0,9625.0,444.0,162.0,9035.0,402.0,149.0,INTELIQUENT INC,2010.0,32.608,199.826,315.527,44.244
1,0001013934,STRAYER EDUCATION INC,2011,0.0,0.0,3153.0,118.0,63.0,562.0,15.0,7.0,STRATEGIC EDUCATION INC,2011.0,106.044,627.434,231.133,188.840
2,0000854775,DIGI INTERNATIONAL INC,2012,1.0,5.0,5363.0,240.0,69.0,3819.0,162.0,56.0,DIGI INTERNATIONAL INC,2012.0,7.615,190.558,293.084,22.227
3,0001158463,JETBLUE AIRWAYS CORP,2011,1.0,4.0,5133.0,220.0,81.0,4192.0,174.0,67.0,JETBLUE AIRWAYS CORP,2011.0,86.000,4504.000,7071.000,5314.000
4,0000059558,LINCOLN NATIONAL CORP,2013,11.0,2.0,8822.0,369.0,113.0,6398.0,253.0,93.0,LINCOLN NATIONAL CORP,2013.0,1244.000,11883.000,236945.000,223493.000


### 4.2 Make the dataframe more organized

In [15]:
merge1=merge.sort_values(['cik','year'],ignore_index=True)
merge1.head()

,cik,company name,year,# of images in cd&a section,# of tables in cd&a section,# of words in cd&a section (including tabular content--old version),# of sentences in cd&a (including tabular content--old version),# of sentences with fls (including tabular content--old version),# of words in cd&a section,# of sentences in cd&a,# of sentences with fls,conm,fyear,ni,sale,at,lt
0,0000001750,AAR CORP,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAR CORP,2006.0,58.660,1061.169,1067.633,573.390
1,0000001750,AAR CORP,2007,0.0,2.0,3824.0,NaN,NaN,3727.0,165.0,64.0,AAR CORP,2007.0,75.144,1384.919,1362.010,776.755
2,0000001750,AAR CORP,2008,0.0,1.0,5005.0,NaN,NaN,4837.0,207.0,81.0,AAR CORP,2008.0,78.651,1423.976,1377.511,720.616
3,0000001750,AAR CORP,2009,0.0,2.0,5560.0,NaN,NaN,5253.0,207.0,79.0,AAR CORP,2009.0,44.628,1352.151,1501.042,754.692
4,0000001750,AAR CORP,2010,0.0,3.0,6034.0,NaN,NaN,5807.0,236.0,96.0,AAR CORP,2010.0,69.826,1775.782,1703.727,868.438


### 4.3 Delete the 'fyear' column as it is the same as the 'year' column

In [16]:
merge2=merge1.drop('fyear',axis=1)
merge2

,cik,company name,year,# of images in cd&a section,# of tables in cd&a section,# of words in cd&a section (including tabular content--old version),# of sentences in cd&a (including tabular content--old version),# of sentences with fls (including tabular content--old version),# of words in cd&a section,# of sentences in cd&a,# of sentences with fls,conm,ni,sale,at,lt
0,0000001750,AAR CORP,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAR CORP,58.660,1061.169,1067.633,573.390
1,0000001750,AAR CORP,2007,0.0,2.0,3824.0,NaN,NaN,3727.0,165.0,64.0,AAR CORP,75.144,1384.919,1362.010,776.755
2,0000001750,AAR CORP,2008,0.0,1.0,5005.0,NaN,NaN,4837.0,207.0,81.0,AAR CORP,78.651,1423.976,1377.511,720.616
3,0000001750,AAR CORP,2009,0.0,2.0,5560.0,NaN,NaN,5253.0,207.0,79.0,AAR CORP,44.628,1352.151,1501.042,754.692
4,0000001750,AAR CORP,2010,0.0,3.0,6034.0,NaN,NaN,5807.0,236.0,96.0,AAR CORP,69.826,1775.782,1703.727,868.438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14989,0001645590,HEWLETT PACKARD ENTERPRISE CO,2016,24.0,2.0,12370.0,NaN,NaN,8165.0,332.0,128.0,HEWLETT PACKARD ENTERPRISE,3161.000,50123.000,79679.000,48161.000
14990,0001646383,CSRA INC.,2016,0.0,6.0,7816.0,NaN,NaN,6547.0,300.0,89.0,CSRA INC,304.000,4993.000,4888.000,4529.000
14991,0001649338,BROADCOM LTD,2016,3.0,6.0,9951.0,NaN,NaN,7099.0,259.0,102.0,NaN,NaN,NaN,NaN,NaN
14992,0001650132,"FOUR CORNERS PROPERTY TRUST, INC.",2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOUR CORNERS PROPERTY TR INC,156.809,124.115,937.151,467.034


In [17]:
merge2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 16 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   cik                                                                  14994 non-null  object 
 1   company name                                                         14994 non-null  object 
 2   year                                                                 14994 non-null  int64  
 3   # of images in cd&a section                                          13434 non-null  float64
 4   # of tables in cd&a section                                          13434 non-null  float64
 5   # of words in cd&a section (including tabular content--old version)  13434 non-null  float64
 6   # of sentences in cd&a (including tabular content--old version)      100 non-null    float64
 7   # of

### 4.4 Drop duplicates

In [18]:
merge3=merge2.drop_duplicates(subset=['cik','company name','year'],keep='first')
merge3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14755 entries, 0 to 14993
Data columns (total 16 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   cik                                                                  14755 non-null  object 
 1   company name                                                         14755 non-null  object 
 2   year                                                                 14755 non-null  int64  
 3   # of images in cd&a section                                          13319 non-null  float64
 4   # of tables in cd&a section                                          13319 non-null  float64
 5   # of words in cd&a section (including tabular content--old version)  13319 non-null  float64
 6   # of sentences in cd&a (including tabular content--old version)      100 non-null    float64
 7   # of